In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

In [3]:
dataset = './hand_sign_history.csv'
model_save_path = './hand_sign_history.hdf5'
tflite_save_path = './hand_sign_history.tflite'

In [27]:
NUM_CLASSES = 3
TIME_STEPS = 16
DIMENSION = 2

def transform_y(y):
  if y == 7:
    return 0
  if y == 10:
    return 1
  if y == 23:
    return 2
  return 3

In [35]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (TIME_STEPS * DIMENSION) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
y_dataset = np.vectorize(transform_y)(y_dataset)
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.8, random_state=RANDOM_SEED)

In [36]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(TIME_STEPS * DIMENSION, )),
    tf.keras.layers.Reshape((TIME_STEPS, DIMENSION), input_shape=(TIME_STEPS * DIMENSION, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(16, input_shape=[TIME_STEPS, DIMENSION]),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_4 (Reshape)         (None, 16, 2)             0         
                                                                 
 dropout_8 (Dropout)         (None, 16, 2)             0         
                                                                 
 lstm_4 (LSTM)               (None, 16)                1216      
                                                                 
 dropout_9 (Dropout)         (None, 16)                0         
                                                                 
 dense_8 (Dense)             (None, 10)                170       
                                                                 
 dense_9 (Dense)             (None, 3)                 33        
                                                                 
Total params: 1419 (5.54 KB)
Trainable params: 1419 (5

In [37]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=300,
    batch_size=128,
    validation_data=(X_test, y_test)
)

In [39]:
model.save(model_save_path, include_optimizer=False)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)

    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)

    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()

    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

In [41]:
run_model = tf.function(lambda x: model(x))
BATCH_SIZE = 1
INPUT_SIZE = TIME_STEPS * DIMENSION
concrete_func = run_model.get_concrete_function(tf.TensorSpec([BATCH_SIZE, INPUT_SIZE], model.inputs[0].dtype))
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [42]:
tflite_quantized_model = converter.convert()
f = open(tflite_save_path, 'wb')
f.write(tflite_quantized_model)
f.close()